In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import requests

Mounted at /content/drive


In [ ]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 8.8 MB/s eta 0:00:00


In [ ]:
from pypdf import PdfReader

In [ ]:
url = "https://drive.google.com/uc?id=13hn0k7A7W_Vpk9yIAiem7t4JHSOoY4lQ"
response = requests.get(url)

with open("cuento_plano_v2.pdf", "wb") as f:
    f.write(response.content)

documento = PdfReader('/content/cuento_plano_v2.pdf')
paguinas = []

for i,page in enumerate(documento.pages):
  txt = page.extract_text() or ""
  paguinas.append(txt)

texto_completo = "\n".join(paguinas)

In [ ]:
texto_completo

'El ultimo faro\nCuento breve - 2025-09-23\nEn un pueblo costero, el faro llevaba decadas encendido, respirando luz en la niebla como\nsi fuera un viejo que sopla sobre un vaso de te. Nadie recordaba cuando se habian quedado\nsin su parpadeo por mas de una noche. Los pescadores juraban que una vez, durante una\ntormenta que partio el cielo como madera humeda, la luz del faro dibujo un camino exacto\nentre las olas y los condujo a puerto. No era un milagro, decian, era su oficio.\nEl farero se llamaba Elias. Caminaba despacio, con una linterna de aceite y una calma que\nparecia prestada del mar en invierno. Su rutina era sencilla: subir los escalones, limpiar\nlos cristales, revisar engranajes, hablar en voz baja con la lampara como si fuera una\ncriatura discreta que respira keroseno y devuelve seguridad.\nUna tarde, llego al pueblo una noticia rara: decian que habia un nuevo sistema de\nnavegacion por satelites, que los barcos podian ver el mundo desde arriba, que ya no haria\nfalta u

In [ ]:
len(texto_completo)

7158

In [ ]:
set_stopwords = set([
        'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una',
        'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre',
        'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo',
        'nos', 'durante', 'todos', 'uno', 'les',  # 50
        'ni',  # 51
        # Palabras adicionales
        "tú", "tus", "te", "ti", "tu", "vos", "vosotros", "vosotras", "ustedes", "su", "sus", "ello", "ellos",
        "ellas", "mío", "mía", "míos", "mías", "tuyo", "tuya", "tuyos", "tuyas", "suyo", "suya", "suyos",
        "suyas", "nuestro", "nuestra", "nuestros", "nuestras",  # 80
        # Contracciones comunes
        "al",  # a + el
        "del"  # de + el
    ])

In [ ]:
def tokenize_v4(text):
    #delimiters = string.whitespace + string.punctuation
    acentos_enye = set([
        193, 201, 205, 211, 218,  # Á, É, Í, Ó, Ú
        225, 233, 237, 243, 250,  # á, é, í, ó, ú
        209, 241,                 # Ñ, ñ
        220, 252                  # Ü, ü
    ])

    list_Mm = {193:225,#32
              201:233,#32
              205:237,#32
              211:243,#32
              218:250,#32
              209:241,#32
              220:252} #32

    set_numbers = set(range(48, 58))  # Números del 0 al 9
    set_abcdario = set(range(65, 91)).union(set(range(97, 123))).union(acentos_enye)  # Letras A-Z, a-z y letras con acentos y ñ


    token = ""
    tokens = [None]*len(text)
    count = 0

    #si la posion del texto es un delimitadir y es la ultima posicion
    #se sustituye
    if ord(text[-1]) not in set_abcdario or ord(text[-1]) not in set_numbers:
        text += '.'
        n = len(text)

    for i,char in enumerate(text):
        code = ord(char)

        # Si es letra mayúscula (A-Z), convierto a minúscula
        if 65 <= code <= 90:
            char = chr(code + 32)
        if code in list_Mm:
            char = chr(list_Mm[code])

        #si el caracter es una letra o numero entonces se almacena en token
        if(ord(char) in set_abcdario or ord(char) in set_numbers):
            token += char
            continue


        if token != "":
            # Limpieza: si hay letras, quitamos dígitos
            has_letter = False
            for c in token:
                if ord(c) in set_abcdario:
                    has_letter = True
                    break

            if has_letter:
                new_token = ""
                for c in token:
                    if ord(c) in set_abcdario:
                        new_token += c
                token = new_token
            # Si no tenía letras, es un número puro y se queda como está

            if token != "":
                if token not in set_stopwords:
                    tokens[count] = token
                    count += 1
            token = ""

    return tokens[0:count]

In [ ]:
def PalabrasUnicas(tokens):
  palabras = []
  for token in tokens:
    if token not in palabras:
      palabras.append(token)

  return palabras

In [ ]:
vector_unic = PalabrasUnicas(tokenize_v4(texto_completo))
print(vector_unic)
print(len(vector_unic))

['ultimo', 'faro', 'cuento', 'breve', '2025', '09', '23', 'pueblo', 'costero', 'llevaba', 'decadas', 'encendido', 'respirando', 'luz', 'niebla', 'si', 'fuera', 'viejo', 'sopla', 'vaso', 'nadie', 'recordaba', 'habian', 'quedado', 'parpadeo', 'mas', 'noche', 'pescadores', 'juraban', 'vez', 'tormenta', 'partio', 'cielo', 'madera', 'humeda', 'dibujo', 'camino', 'exacto', 'olas', 'condujo', 'puerto', 'era', 'milagro', 'decian', 'oficio', 'farero', 'llamaba', 'elias', 'caminaba', 'despacio', 'linterna', 'aceite', 'calma', 'parecia', 'prestada', 'mar', 'invierno', 'rutina', 'sencilla', 'subir', 'escalones', 'limpiar', 'cristales', 'revisar', 'engranajes', 'hablar', 'voz', 'baja', 'lampara', 'criatura', 'discreta', 'respira', 'keroseno', 'devuelve', 'seguridad', 'tarde', 'llego', 'noticia', 'rara', 'habia', 'nuevo', 'sistema', 'navegacion', 'satelites', 'barcos', 'podian', 'ver', 'mundo', 'arriba', 'haria', 'falta', 'alcaldia', 'convoco', 'reunion', 'vinieron', 'comerciantes', 'turistas', 'pas

In [ ]:
#ONE Hot Encoding
def ohe(vector):
  dim=len(vector)
  vector_hot= []
  DIC={}
  for i in range(dim):
    vector_hot.append([])
    for j in range(dim):
      if i==j:
        vector_hot[i].append(1)
      else:
        vector_hot[i].append(0)

    DIC[vector[i]]=vector_hot[i]
  return DIC

In [ ]:
#vector_unic = 'Marco va reprobar alumnos PLN'

In [ ]:
one_hot = ohe(vector_unic)
print(one_hot)

{'ultimo': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0